In [1]:
import json
import os
import sys

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
os.environ.pop('TF_CONFIG', None)

In [4]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

In [5]:
import tensorflow as tf

In [6]:
%%writefile mnist_setup.py

import os
import tensorflow as tf
import numpy as np

def mnist_dataset(batch_size):
  (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
  # The `x` arrays are in uint8 and have values in the [0, 255] range.
  # You need to convert them to float32 with values in the [0, 1] range.
  x_train = x_train / np.float32(255)
  y_train = y_train.astype(np.int64)
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(60000).repeat().batch(batch_size)
  return train_dataset

def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.InputLayer(input_shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
      metrics=['accuracy'])
  return model

Overwriting mnist_setup.py


In [7]:
import mnist_setup

batch_size = 64
single_worker_dataset = mnist_setup.mnist_dataset(batch_size)
single_worker_model = mnist_setup.build_and_compile_cnn_model()
single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)

Epoch 1/3


2022-03-26 14:19:44.213616: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-26 14:19:44.213668: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lisy09-win): /proc/driver/nvidia/version does not exist
2022-03-26 14:19:44.214122: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


70/70 [==============================] - 1s 10ms/step - loss: 2.2834 - accuracy: 0.1397
Epoch 2/3
70/70 [==============================] - 1s 10ms/step - loss: 2.2187 - accuracy: 0.2875
Epoch 3/3
70/70 [==============================] - 1s 10ms/step - loss: 2.1464 - accuracy: 0.4705


In [8]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

In [9]:
os.environ['GREETINGS'] = 'Hello TensorFlow!'

In [10]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO


In [11]:
with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()

In [12]:
%%writefile multi_worker_main.py

import os
import json

import tensorflow as tf
import mnist_setup

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

strategy = tf.distribute.MultiWorkerMirroredStrategy()

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = mnist_setup.mnist_dataset(global_batch_size)

with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()


multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)

Overwriting multi_worker_main.py


In [13]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [14]:
# first kill any previous runs
%killbgscripts

All background processes were killed.


In [15]:
%%bash --bg
python multi_worker_main.py &> job_0.log

In [16]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [17]:
%%bash
python multi_worker_main.py

Epoch 1/3
70/70 [==============================] - 6s 46ms/step - loss: 2.2830 - accuracy: 0.1598
Epoch 2/3
70/70 [==============================] - 3s 43ms/step - loss: 2.2449 - accuracy: 0.3084
Epoch 3/3
70/70 [==============================] - 3s 39ms/step - loss: 2.1980 - accuracy: 0.4212


2022-03-26 14:19:49.038075: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-26 14:19:49.038132: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lisy09-win): /proc/driver/nvidia/version does not exist
2022-03-26 14:19:49.038935: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-26 14:19:49.051807: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> localhost:12345, 1 -> localhost:23456}
2022-03-26 14:19:49.052381: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:437] Started server with

In [18]:
cat job_0.log

2022-03-26 14:19:48.860916: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-26 14:19:48.860983: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lisy09-win): /proc/driver/nvidia/version does not exist
2022-03-26 14:19:48.861713: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-26 14:19:48.875195: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> localhost:12345, 1 -> localhost:23456}
2022-03-26 14:19:48.875793: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:437] Started server with